In [0]:
%run /eReport/PkgGeneral/PrcGeneral

In [0]:
import pandas as pd
from datetime import datetime
from datetime import timedelta

class PrcRegistraClasificacionRefinanciada():
  
  def __init__(self, id_solicitud):
      self.id_solicitud = id_solicitud
      
      #Obteniendo parametros
      parametros = PrcGeneral(id_solicitud = self.id_solicitud).obtenerParametros()
      self.fecha_proceso = parametros['fecha_proceso']  
      self.fecha_proceso_str = self.fecha_proceso.strftime('%Y-%m-%d')
      
      #Fecha de actualizacion
      fecha_actualizacion = datetime.now()
      self.fecha_actualizacion_str = fecha_actualizacion.strftime("%Y-%m-%d %H:%M:%S")
      
      #Conexion
      connection = PrcGeneral().databaseConnect()
      self.connection = connection
      
      cursor = self.connection.cursor()
      self.cursor = cursor
      
  def registrarClasificacionRefinanciada(self):

      #Obteniendo tablas
      select_ClienteProducto = ("SELECT * FROM CLIENTE_PRODUCTO")
      sql_ClienteProducto = pd.read_sql_query(select_ClienteProducto, self.connection)
      df_ClienteProducto = pd.DataFrame(sql_ClienteProducto)

      select_Refinanciamiento = ("SELECT * FROM REFINANCIAMIENTO WHERE FECHA_PROCESO = '" + self.fecha_proceso_str + "'")
      sql_Refinanciamiento = pd.read_sql_query(select_Refinanciamiento, self.connection)
      df_Refinanciamiento = pd.DataFrame(sql_Refinanciamiento)

      select_Cliente = ("SELECT * FROM CLIENTE WHERE FECHA_PROCESO = '" + self.fecha_proceso_str + "'")
      sql_Cliente = pd.read_sql_query(select_Cliente, self.connection)
      df_Cliente = pd.DataFrame(sql_Cliente)
      df_Cliente = df_Cliente.rename(columns={'ID': 'CLIENTE'})    

      df_dias_atraso = df_ClienteProducto.merge(df_Refinanciamiento, on='CLIENTE_DEUDOR_PRODUCTO', how='inner')
      df_dias_atraso = df_dias_atraso.merge(df_Cliente, on='CLIENTE', how='inner')
      df_dias_atraso = df_dias_atraso.loc[(df_dias_atraso.TIPO_CREDITO==2)&(df_dias_atraso.SITUACION_CREDITO=='N')]
      df_dias_atraso = df_dias_atraso[['FECHA_PROCESO_x','ID_y','DIAS_ATRASO_x','CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO_x']]
      df_dias_atraso['FECHA_ACTUALIZACION'] = self.fecha_actualizacion_str    

      #Reproceso
      delete_RefinanciamientoDias = "DELETE REFINANCIAMIENTO_DIAS WHERE FECHA_PROCESO = '" + self.fecha_proceso_str + "'"
      self.cursor.execute(delete_RefinanciamientoDias)
      self.connection.commit()

      #Actualizando dias de atraso y clasificacion cliente
      insert_refinanciamiento_dias = "INSERT INTO [dbo].[REFINANCIAMIENTO_DIAS](FECHA_PROCESO,REFINANCIAMIENTO,DIAS_ATRASO,CLASIFICACION_DEUDOR,FECHA_ACTUALIZACION) VALUES (?,?,?,?,?)"
      self.cursor.fast_executemany = True
      values = df_dias_atraso.values.tolist()
      if values:
        self.cursor.executemany(insert_refinanciamiento_dias, values)
        self.connection.commit()    
      self.cursor.close()